In [67]:
import pandas as pd
import numpy as np

df = pd.read_csv("flavors_of_cacao.csv")
df.head()

,Company (Maker-if known),Specific Bean Origin or Bar Name,REF,Review Date,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru


In [68]:
df["Broad Bean\nOrigin"].value_counts().head(20)

Venezuela             214
Ecuador               193
Peru                  165
Madagascar            145
Dominican Republic    141
                       73
Nicaragua              60
Brazil                 58
Bolivia                57
Belize                 49
Papua New Guinea       42
Colombia               40
Costa Rica             38
Vietnam                38
Tanzania               34
Ghana                  33
Trinidad               33
Mexico                 30
Guatemala              28
Hawaii                 28
Name: Broad Bean\nOrigin, dtype: int64

In [69]:
import plotly.plotly as py
%matplotlib inline


In [95]:
counts = pd.DataFrame(df["Broad Bean\nOrigin"].value_counts())
counts = counts[counts["Broad Bean\nOrigin"] > 10]
counts = counts[counts["Broad Bean\nOrigin"] != 73]
counts

,Broad Bean Origin
Venezuela,214
Ecuador,193
Peru,165
Madagascar,145
Dominican Republic,141
Nicaragua,60
Brazil,58
Bolivia,57
Belize,49
Papua New Guinea,42


In [154]:
ratings = df.groupby(['Broad Bean\nOrigin'], as_index=False).mean()
ratings["Rating"] = np.round(ratings["Rating"], decimals=2)

ratings = ratings.drop(["REF", "Review\nDate"], axis=1)
ratings.head()

,Broad Bean Origin,Rating
0,"Africa, Carribean, C. Am.",2.75
1,Australia,3.25
2,Belize,3.23
3,Bolivia,3.20
4,Brazil,3.28


In [155]:
reviews = counts.join(ratings.set_index("Broad Bean\nOrigin"))
reviews["Country"] = reviews.index
reviews.head()

,Broad Bean Origin,Rating,Country
Venezuela,214,3.25,Venezuela
Ecuador,193,3.13,Ecuador
Peru,165,3.14,Peru
Madagascar,145,3.27,Madagascar
Dominican Republic,141,3.21,Dominican Republic


In [156]:
import pycountry

def get_country(name):
#     print(name)
    if name == "Venezuela" or name == "Bolivia":
        name = "Venezuela, Bolivarian Republic of"
    elif name == "Vietnam":
        name = "Viet Nam"
    elif name == "Tanzania":
        name = "Tanzania, United Republic of"
    elif name == "Trinidad":
        name = "Trinidad and Tobago"
    elif name == "Hawaii":
        name = "United States Minor Outlying Islands"
    elif name == "Domincan Republic":
        name = "Dominican Republic"
        
    return pycountry.countries.get(name=name).alpha_3

reviews["CODE"] = reviews["Country"].apply(get_country)
reviews.head()

,Broad Bean Origin,Rating,Country,CODE
Venezuela,214,3.25,Venezuela,VEN
Ecuador,193,3.13,Ecuador,ECU
Peru,165,3.14,Peru,PER
Madagascar,145,3.27,Madagascar,MDG
Dominican Republic,141,3.21,Dominican Republic,DOM


In [222]:
import colorlover as cl
ryb = cl.scales['5']['seq']["Reds"]; ryb
scale = np.asarray(list(range(1, len(ryb) + 1)))/5
# colorscale = list(zip(scale, ryb))

full = []
for i in range(len(scale)):
    print([scale[i], ryb[i]])
    full.append([scale[i], ryb[i]])
    
full

[0.20000000000000001, 'rgb(254,229,217)']
[0.40000000000000002, 'rgb(252,174,145)']
[0.59999999999999998, 'rgb(251,106,74)']
[0.80000000000000004, 'rgb(222,45,38)']
[1.0, 'rgb(165,15,21)']


[[0.20000000000000001, 'rgb(254,229,217)'],
 [0.40000000000000002, 'rgb(252,174,145)'],
 [0.59999999999999998, 'rgb(251,106,74)'],
 [0.80000000000000004, 'rgb(222,45,38)'],
 [1.0, 'rgb(165,15,21)']]

In [220]:
data = [ dict(
        type = 'choropleth',
        locations = reviews['CODE'],
        z = reviews['Rating'],
        text = reviews['Country'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Rating'),
      ) ]

layout = dict(
    title = 'Ratings of Cocoa Sourced frm Around the World',
    geo = dict(
        showframe = True,
        showcoastlines = True,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )